# This script runs a prototype feed-forward prototype of the neural network that will be run on a zybo fpga. The script will produce a c-compatible file with values representations stored in (hex??). 

### Todo
- [ ] convert decimal to binary.
- [ ] create a binary multiplier function (optimal?)
- [ ] choose a prototype architecture and store relevant results in a certain format.
- [ ] Decide how software/hardware fixed-point results will be compared to verify correct implementation on hardware? Do i need to do it in C on the Zynq?

### Outstanding Questions

- How should I implement the multiplier in python?

## How to choose an architecture?
my bit representation is S4.27 -> meaning I have a two's complement sign bit, 4 integer bits and 27 fractional bits.

What does this mean?
- fractional resolution is: 0.00000000372529029846
    - I can represent 7 fractional digits accurately?
Biggest magnitude number I can represent is +/- (16 - 0.00000000372529029846) Right?
This is probably overkill. Would be worthwhile looking at reviews of fixed point precision compared to various metrics.

I will stick with this convention for this milestone, but can try different conventions for future milestones.

In [2]:
import numpy as np
from fxpmath import Fxp
import sys
import os
import glob

In [3]:
for entry in os.listdir(os.path.join(os.getcwd(),"bin_files")):
    # print(entry)
    pass

In [33]:
%run plnn_validation.py True

['01010111101100100110110100011001', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00001101111100010111111010000001', '00101111100101110001100000001011', '00000000000000000000000000000000', '00000000000000000000000000000000', '01000000001100001111010010101001']
[10.96212215718807, -2.856499455477124, -13.964644359637491, -6.808845697520238, -3.36392988279792, 1.7429170691671678, 5.948776329503073, -7.90305153311236, -1.6426185502395225, 8.023904154140283]
['00010001101000001111011110011111', '00000001101000111001100000111111', '00000000000000000000000000000000', '00001011100110001101000110110100', '00000000000000000000000000000000', '01101000100010110101101011000100', '01011111000111010100101000001010', '01111100000110001111011011001000', '00001001110101000011000110010000', '00000000000000000000000000000000']
[2.2035973031988663, 0.20488023421485702, -8.298707189710214, 1.449618733997708,

In [287]:
%run create_img_h_file.py

FileNotFoundError: [Errno 2] No such file or directory: 'C:/git_repos/mnist_neuralnet/scripts/MNIST_single_test_img1.txt'

# Workbench

In [1]:
import numpy as np
from fxpmath import Fxp
import sys
import os
import glob

In [304]:
# Thank you Chatgpt!!!
def twos_complement(bin_str):
    """Returns the two's complement of a binary string."""
    # Invert the bits
    inverted = ''.join('1' if bit == '0' else '0' for bit in bin_str)
    # Add 1 to the inverted binary string
    carry = 1
    result = list(inverted)
    
    for i in range(len(inverted) - 1, -1, -1):
        if result[i] == '1' and carry == 1:
            result[i] = '0'
        elif result[i] == '0' and carry == 1:
            result[i] = '1'
            carry = 0
        # No carry to add; break early
        if carry == 0:
            break
    
    return ''.join(result)

def add_binary(bin_str1, bin_str2):
    """Adds two binary strings and returns the result as a binary string, 
    handling overflow by setting the result to the max/min value depending on overflow direction."""
    max_len = max(len(bin_str1), len(bin_str2))
    bin_str1 = bin_str1.zfill(max_len)
    bin_str2 = bin_str2.zfill(max_len)
    
    carry = 0
    result = []
    
    # Perform bitwise addition from LSB to MSB
    for i in range(max_len - 1, -1, -1):
        bit_sum = carry + int(bin_str1[i]) + int(bin_str2[i])
        result.append(str(bit_sum % 2))
        carry = bit_sum // 2
    
    result = ''.join(result[::-1])  # Reverse the result to get the correct order
    
    # Check for overflow
    sign1 = bin_str1[0]  # Sign bit of the first number
    sign2 = bin_str2[0]  # Sign bit of the second number
    result_sign = result[0]  # Sign bit of the result

    # Overflow occurs if both numbers have the same sign but the result has a different sign
    if sign1 == sign2 and sign1 != result_sign:
        if sign1 == '0':  # Positive overflow
            result = '0' + '1' * (max_len - 1)  # Max positive value: 011...111
            raise Exception(f"OVERFLOW")
        else:  # Negative overflow
            result = '1' + '0' * (max_len - 1)  # Max negative value: 100...000
            raise Exception(f"OVERFLOW")
    
    return result


def manual_binary_multiply(bin_str1, bin_str2):
    """Multiplies two binary strings in two's complement format manually."""
    # Check for sign and convert to positive if necessary
    is_negative1 = bin_str1[0] == '1'
    is_negative2 = bin_str2[0] == '1'
    
    if is_negative1:
        bin_str1 = twos_complement(bin_str1)
    if is_negative2:
        bin_str2 = twos_complement(bin_str2)

    # Perform binary multiplication (manual)
    len1 = len(bin_str1)
    len2 = len(bin_str2)
    result = '0' * (len1 + len2)
    
    for i in range(len2 - 1, -1, -1):
        if bin_str2[i] == '1':
            # Shift bin_str1 by (len2 - 1 - i) and add to the result
            shifted_bin_str1 = bin_str1 + '0' * (len2 - 1 - i)
            result = add_binary(result, shifted_bin_str1.zfill(len1 + len2))
    
    # Determine the sign of the result
    if is_negative1 != is_negative2:
        result = twos_complement(result.zfill(len1 + len2))

    # Truncate the result to the appropriate length (len1 + len2 bits)
    return result[-(len1 + len2):]

# Example usage:
bin_str1 = "1101101011010100010110101"  # -3 in 4-bit two's complement
bin_str2 = "1010010111010101010101001"  # -6 in 4-bit two's complement

result = manual_binary_multiply(bin_str1, bin_str2)
print(result)  # 8-bit result in two's complement binary format


00001101000101111000101000111110111001000101111101


In [305]:
def convert32_64(bin_str1):
    bin_str2 = ['0']*64
    if bin_str1[0] == '1': # this means the number is negative
        bin_str2[:5] = '1'*5
        
    bin_str2[5:len(bin_str1)+5] = bin_str1
    bin_str2 = ''.join(bin_str2)
        
    return bin_str2

def convert64_32(bin_str1):
    if bin_str1[0] == '1':
        bin_str2 = '0'*32
    else:
        bin_str2 = bin_str1[5:32+5]
        
    return bin_str2

In [340]:
gen_output_files = False

In [341]:

    # Define the folder for the binary files
    hex_folder = "hex_files"
    txt_folder = "txt_files"
    bin_folder = "bin_files"
    h_folder = "header_files"
    folders = [hex_folder, txt_folder, bin_folder, h_folder]
    
    x32 = Fxp(-7.25, dtype='S5.27')
    x64 = Fxp(-7.25, dtype='S10.54')
    num_layers = 2  # Example: 2 layers
    neurons_per_layer = [10,10]  # Example: 3 neurons in layer 1, 2 neurons in layer 2
    input_size = 784
    weights = []
    bias = []
    img = [0 for k in range(input_size)]
    if gen_output_files:
        for folder in folders:
            # Create the folder if it doesn't exist
            os.makedirs(folder, exist_ok=True)
            files = glob.glob(os.path.join(folder, "*"))
            for f in files:
                try:
                    os.remove(f)  # Remove each file
                except Exception as e:
                    print(f"Error deleting file {f}: {e}")
        img = (np.random.rand(1,input_size) * 2) -1
    
        with open(os.path.join(txt_folder, "img.txt"), "w") as file:
            for i in range(input_size):
                file.write(str(float(x32(img[0,i])))+"\n")
        with open(os.path.join(bin_folder, "img.mif"), "w") as file:
            for i in range(input_size):
                file.write(str(x32(img[0,i]).bin())+"\n")
        with open(os.path.join(hex_folder, "img.mif"), "w") as file:
            for i in range(input_size):
                file.write(str(x32(img[0,i]).hex())+"\n")
        
        for layer in range(num_layers):
            
            input_size_for_layer = input_size if layer == 0 else neurons_per_layer[layer - 1]
            weights.append((np.random.rand(input_size_for_layer, neurons_per_layer[layer]) * 2) - 1)
            bias.append((np.random.rand(neurons_per_layer[layer]) * 2) - 1)
            
            with open(os.path.join(txt_folder, f"bias_{layer}.txt"), "w") as file:
                for i in range(neurons_per_layer[layer]):
                    file.write(str(float(x32(bias[layer][i]))) + "\n")
            with open(os.path.join(bin_folder, f"bias_{layer}.mif"), "w") as file:
                for i in range(neurons_per_layer[layer]):
                    file.write(str(x32(bias[layer][i]).bin()) + "\n")
            with open(os.path.join(hex_folder, f"bias_{layer}.mif"), "w") as file:
                for i in range(neurons_per_layer[layer]):
                    file.write(str(x32(bias[layer][i]).hex()) + "\n")
                    
            for neuron in range(neurons_per_layer[layer]):
                    
                with open(os.path.join(txt_folder, f"weight_{layer}_{neuron}.txt"), "w") as file:
                    for i in range(input_size_for_layer):
                        file.write(str(float(x32(weights[layer][i, neuron]))) + "\n")
                with open(os.path.join(bin_folder, f"weight_{layer}_{neuron}.mif"), "w") as file:
                    for i in range(input_size_for_layer):
                        file.write(str(x32(weights[layer][i, neuron]).bin()) + "\n")
                with open(os.path.join(hex_folder, f"weight_{layer}_{neuron}.mif"), "w") as file:
                    for i in range(input_size_for_layer):
                        file.write(str(x32(weights[layer][i, neuron]).hex()) + "\n")
                        
        temp_bias = list(bias[0]) + list(bias[1])
        with open(os.path.join(h_folder, "img.h"), "w") as file:
            file.write("#ifndef IMG_H\n")
            file.write("#define IMG_H\n\n")
            
            file.write("uint32_t bin_img[784] = { ")
            for i in range(input_size):
                # Convert the value to an unsigned char representation (assuming the range is normalized between -1 and 1)
                file.write( '0b'+str(x32(img[0,i]).bin())+", " if i < input_size -1 else '0b'+str(x32(img[0,i]).bin()) )
            file.write(" };\n\n")

            for i in range(2):
                for j in range(10):
                    file.write(f"uint32_t weights_{i}_{j}[{input_size}] =" + " { "  + ''.join([f"0b{x32(k).bin()}, " for k in weights[i][:-1,j]]) + f"0b{x32(weights[i][-1,j]).bin()}" + " };\n\n")
                    
            file.write("uint32_t bias [10] = {" )
            for i in range(2):
                for j in range(10):
                    if (i*10)+j < 19:
                        file.write(''.join(f"0b{x32(temp_bias[(i*10)+j]).bin()}, " ))
            file.write(f"0b{x32(temp_bias[-1]).bin()}" + " };\n\n")
                    
                    
    
            
            file.write("#endif // IMG_H\n")
    weights = []
    bias = []
    img = [0 for k in range(input_size)]
    # Read img for the current layer
    with open(os.path.join(bin_folder, f"img.mif"), "r") as file:
        lines = file.readlines()
        for i in range(input_size):
            img[i] = lines[i].strip()
    
    # Reading weights and biases from files for each layer
    for layer in range(num_layers):
        input_size_for_layer = input_size if layer == 0 else neurons_per_layer[layer - 1]
        
        # Initialize weights and bias for the current layer
        weights_layer = np.zeros((input_size_for_layer, neurons_per_layer[layer]),dtype=( np.str_, 32))
        bias_layer = np.zeros(neurons_per_layer[layer],dtype=( np.str_, 32))
        
        # Read biases for the current layer
        with open(os.path.join(bin_folder, f"bias_{layer}.mif"), "r") as file:
            lines = file.readlines()
            for i in range(neurons_per_layer[layer]):
                bias_layer[i] = lines[i].strip()
        bias.append(bias_layer)
    
        # Read weights for the current layer
        for neuron in range(neurons_per_layer[layer]):
            with open(os.path.join(bin_folder, f"weight_{layer}_{neuron}.mif"), "r") as file:
                lines = file.readlines()
                for i in range(input_size_for_layer):
                    weights_layer[i, neuron] = lines[i].strip()
                    
        weights.append(weights_layer)
    
    # Feedforward fixed-point calculation for each layer
    a_tdata = []
    a_tdata_float = []
    for layer in range(num_layers):
        input_size_for_layer = input_size if layer == 0 else neurons_per_layer[layer - 1]
        input_data = img if layer == 0 else a_tdata[layer - 1]
        layer_output = []
        layer_output_float = []
        for j in range(neurons_per_layer[layer]):
            b = bias[layer][j]
            acc = convert32_64(b)
            
            b_float = float(x32(''.join(('0b',bias[layer][j]))))
            acc_float = b_float
    
            if  float(x32(''.join(('0b',b)))) != b_float:
                raise Exception(f"BIAS %s, %d, are not equal at layer {layer}, neuron {j} and input value {i}", float(x64(''.join(('0b',b)))), b_float)
            
            for i in range(input_size_for_layer):
                w = weights[layer][i,j]
                w, x = weights[layer][i, j], input_data[i]
                p = manual_binary_multiply(w, x)
                # if add_binary(acc,p) == 'fail': 
                #     raise Except(f"ERROR OVERFLOW at {layer, j, i}")
                acc = add_binary(acc, p)

                if len(input_data[i]) != 32:
                    print(i)
                    print(input_data[i])
                    
                w_float, x_float = float(x32(''.join(('0b',weights[layer][i, j])))), float(x32(''.join(('0b',input_data[i]))))
                p_float = w_float * x_float
                acc_float = acc_float + p_float
    
                val1 = float(x64(''.join(('0b',acc))))
                val2 = float(x32(''.join(('0b',w))))
                val3 = float(x32(''.join(('0b',x))))
                val4 = float(x64(''.join(('0b',p))))
                
                if  val2 != w_float:
                    raise Exception(f"WEIGHTS %s, %d, are not equal at layer {layer}, neuron {j} and input value {i}", val2, w_float)
                if  val3 != x_float:
                    raise Exception(f"img %s, %d, are not equal at layer {layer}, neuron {j} and input value {i}", val3, x_float)
                if  val4 != p_float:
                    raise Exception(f"SUM %s, %d, are not equal at layer {layer}, neuron {j} and input value {i}", val4, p_float)
                if  abs(val1 - acc_float) > 0.0000001: # This verifies that the fixed point calculation agrees with the floating point calculation within 7 decimal places...
                    raise Exception(f" ACCUMULATE %s, %d, are not equal at layer {layer}, neuron {j} and input value {i}", val1, acc_float)
                
            layer_output.append(convert64_32(acc))
            
            layer_output_float.append(acc_float)
        
        a_tdata.append(layer_output)
        
        a_tdata_float.append(layer_output_float)
        
        with open(os.path.join(bin_folder, f"output_layer_{layer}.mif"), "w") as file:
            for i in range(neurons_per_layer[layer]):
                file.write(layer_output[i] + "\n")
        with open(os.path.join(hex_folder, f"output_layer_{layer}.mif"), "w") as file:
            for i in range(neurons_per_layer[layer]):
                file.write(x32(''.join(('0b',layer_output[i]))).hex() + "\n")
         
        with open(os.path.join(txt_folder, f"output_layer_{layer}.txt"), "w") as file:
            for i in range(neurons_per_layer[layer]):
                file.write(str(layer_output_float[i]) + "\n")
                
    for layer in range(num_layers):
        print(a_tdata[layer], end='\n')
        
        print(a_tdata_float[layer], end='\n')
    
    print("Success!")

0000000010101111111101010001111001010011110010010011001000110100
['00010011101101011011101101111101', '00000000000000000000000000000000', '00001010010111100000000101111001']
[2.463736518964177, -15.50874805773417, 1.2959012477506748]
['00010101111111101010001111001010']
[2.749335843870344]
Success!


In [339]:
print("uint32_t img[784] = { "  + ''.join([f"0b{k}, " for k in img[:-1]]) + f"0b{img[-1]}" + " };")
    

uint32_t img[784] = { 0b11111011011010110011001010111111, 0b00000001000001111101100100001010, 0b00000010100101001101101001110100, 0b11111111101101101111110100110111, 0b00000100111111100001110100101110, 0b11111100010000010000001011101001, 0b11111100100011001000100101100100, 0b11111011001011101001100001011100, 0b11111110110001111111110111000001, 0b11111101011101101010010111100010, 0b11111111011111100011101110101110, 0b11111010111101010011111110110001, 0b00000101100100110111001101010000, 0b00000001001111100100110001111000, 0b00000000001001100110010010010010, 0b00000101100010101000101000001111, 0b11111111111101011101110000001110, 0b11111001100101111111110100101001, 0b00000101101011100100101011100110, 0b11111010011101011000000000100110, 0b11111110010110010101001010101011, 0b00000010111000101001100101110010, 0b00000100011001110101011100111001, 0b00000101000110101011100010011101, 0b00000110010111101110000100011011, 0b11111001011100111010000111111100, 0b11111000010010100011100111000010, 0b1111

In [338]:
img

['11111011011010110011001010111111',
 '00000001000001111101100100001010',
 '00000010100101001101101001110100',
 '11111111101101101111110100110111',
 '00000100111111100001110100101110',
 '11111100010000010000001011101001',
 '11111100100011001000100101100100',
 '11111011001011101001100001011100',
 '11111110110001111111110111000001',
 '11111101011101101010010111100010',
 '11111111011111100011101110101110',
 '11111010111101010011111110110001',
 '00000101100100110111001101010000',
 '00000001001111100100110001111000',
 '00000000001001100110010010010010',
 '00000101100010101000101000001111',
 '11111111111101011101110000001110',
 '11111001100101111111110100101001',
 '00000101101011100100101011100110',
 '11111010011101011000000000100110',
 '11111110010110010101001010101011',
 '00000010111000101001100101110010',
 '00000100011001110101011100111001',
 '00000101000110101011100010011101',
 '00000110010111101110000100011011',
 '11111001011100111010000111111100',
 '11111000010010100011100111000010',
 

In [334]:
print("unsigned char weights_1_0[784] = { "  + ''.join([f"0b{k}, " for k in weights[1][:-1,0]]) + f"0b{weights[1][-1,2]}" + " };")
    

unsigned char weights_1_0[784] = { 0b00000100111010010011010111110111, 0b00000101010000011000111011100011, 0b00000010100110110101001100100001 };


In [337]:
bias[1]

array(['00000110100001000110101010101110'], dtype='<U32')

In [329]:
''.join([f"0b{k}, " for k in weights[0][:-1,0]])

'0b11111011101111010111101111000101, 0b00000011000011010000000101001011, 0b11111110101010100011011100100010, 0b11111101111000001100111000100111, 0b11111110100101010001111011000111, 0b00000111111010001111101011000000, 0b11111110001000110001011110101110, 0b00000111100011100101011111111001, 0b11111100011010010011011110111110, 0b11111000001110000000101101010000, 0b11111101111010111001100000101100, 0b00000110111011100100010010111011, 0b11111000001110110000101111011010, 0b00000100111011011100011010000010, 0b00000101001100011011010011011110, 0b11111101011100000111101001011101, 0b00000001011100110011110010110111, 0b11111010110011000001001001100111, 0b11111011110100101111110110011110, 0b11111001011001110001001000000111, 0b00000111000010110011010000011000, 0b11111010101110000110011011110010, 0b11111000111111110111110011011001, 0b00000010001010101001111011100100, 0b11111001100000001100000000101001, 0b11111001110010100010110011001101, 0b11111100001100100010110001101101, 0b0000011111000100101000000